In [3]:
import numpy as np
import cv2
import math
import time 
net=cv2.dnn.readNet("yolov3.weights","yolov3.cfg")
classes=[]
with open("coco.names","r") as f:
    classes=[line.strip() for line in f.readlines()]

In [5]:
layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
np.random.uniform(0, 255, size=(len(classes), 3))

array([[137.76977669,  97.32838915, 146.36350813],
       [ 79.62603301,  66.33603997, 253.89384461],
       [210.928386  ,  19.03684423,  79.33674133],
       [  8.09240388, 165.5001493 ,  55.18893384],
       [  3.71917588,  80.55131307,  43.54276195],
       [193.48309225, 219.61791545, 217.28790003],
       [ 44.03722855,  17.12952026,  80.72763382],
       [201.92280821, 102.52014388,  67.27109672],
       [124.9867377 , 117.31210726, 224.6371464 ],
       [ 60.09305922, 104.96481533, 179.70276756],
       [179.19951919,  18.99421464, 213.2641657 ],
       [246.44297199, 208.15294131,   5.50562681],
       [189.65476313,   6.56984675,  89.20081054],
       [234.99187   ,   5.62380189,  25.26654386],
       [ 39.383866  , 112.87735394, 233.13910253],
       [207.24147989, 132.39273181,  54.34276754],
       [  2.67871745,  43.84368714,  90.63411081],
       [ 33.69600968, 205.26738707,  33.30529277],
       [241.07079777, 151.2715041 , 250.99677752],
       [ 53.96106672, 229.09548

In [40]:
cap=cv2.VideoCapture('people2.mp4')
def calibrated_dist(p1,p2):
    return((p1[0]-p2[0])**2+(p1[1]-p2[1])**2)**0.5
def isclose(p1,p2):
    c_d=calibrated_dist(p1,p2)
    calib = (p1[1] + p2[1]) / 2
    if 0<c_d<0.15*calib:
        return 1
    elif 0<c_d<0.2*calib:
        return 2
    else:
        return 0
height, width=None,None
while(cap.isOpened()):
    ret,img=cap.read()
    print(ret)
    if not ret:
        break
    if width is None or height is None:
        height,width =img.shape[:2]
        q=width

    img=img[0:height,0:q]
    height,width=img.shape[:2]
    blob = cv2.dnn.blobFromImage(img, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    start=time.time()
    outs=net.forward(output_layers)
    end=time.time()
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores=detection[5:]
            class_id=np.argmax(scores)
            confidence=scores[class_id]
            if confidence>0.5:
                center_x=int(detection[0]*width)
                center_y=int(detection[1]*height)
                w=int(detection[2]*width)
                h=int(detection[2]*height)
                x=int(center_x-w/2)
                y=int(center_y-h/2)
                boxes.append([x,y,w,h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes=cv2.dnn.NMSBoxes(boxes,confidences,0.5,0.5)
    font=cv2.FONT_HERSHEY_SIMPLEX
    if len(indexes)>0:
        status=list()
        idf=indexes.flatten()
        close_pair=list()
        s_close_pair=list()
        center=list()
        dist=list()
        persons=list()
        for i in idf:
            (x,y)=(boxes[i][0],boxes[i][1])
            (w,h)=(boxes[i][2],boxes[i][3])
            label=str(classes[class_ids[i]])
            if label=='person':
                cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
                cv2.putText(img,label,(x,y+30),font,1,(255,0,0),2)
                center.append([int(x+w/2),int(y+h/2)])
                persons.append(i)
                status.append(0)
        print("idf,person",idf,persons)
        for i in range(len(center)):
            for j in range(len(center)):
                g=isclose(center[i],center[j])
                if g==1:
                    close_pair.append([center[i],center[j]])
                    status[i]=1
                    status[j]=1
                elif g==2:
                    s_close_pair.append([center[i],center[j]])
                    if status[i]!=1:
                        status[i]=2
                    if status[j]!=1:
                        status[j]=2
        total_p=len(center)
        low_risk_p=status.count(2)
        high_risk_p=status.count(1)
        safe_p=status.count(0)
        kk=0
        for i in persons:
            
            sub_img = img[10:170, 10:width-10]
            black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 0
            res = cv2.addWeighted(sub_img, 0.77, black_rect, 0.23, 1.0)
            img[10:170, 10:width-10] = res
    
            # Main Title
            cv2.putText(img, "Social Distancing Detection - During Pandemic", (255, 45), font, 1, (255, 255, 255), 2)
            window_name = "Social Distancing Detection"
            cv2.namedWindow(window_name, cv2.WND_PROP_FULLSCREEN)
            cv2.setWindowProperty(window_name, cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
            cv2.putText(img, "Anshika Gupta", (450, 200), font, 1, (0, 255, 255), 2)

            # Bounding boxes for information
            cv2.rectangle(img, (20, 60), (625, 160), (170, 170, 170), 2)
            cv2.putText(img, "Connecting lines show closeness among people.", (45, 80), font, 0.6, (255, 255, 0), 1)
            cv2.putText(img, "YELLOW: CLOSE", (45, 110), font, 0.5, (0, 255, 255), 1)
            cv2.putText(img, "RED: VERY CLOSE", (45, 130), font, 0.5, (0, 0, 255), 1)
    
            cv2.rectangle(img, (675, 60), (width-20, 160), (170, 170, 170), 2)
            cv2.putText(img, "Bounding boxes show the level of risk to the person.", (685, 80), font, 0.6, (255, 255, 0), 1)
            cv2.putText(img, "Dark RED: HIGH RISK", (685, 110), font, 0.5, (0, 0, 150), 1)
            cv2.putText(img, "ORANGE: LOW RISK", (685, 130), font, 0.5, (0, 120, 255), 1)
            cv2.putText(img, "GREEN: SAFE", (685, 150), font, 0.5, (0, 255, 0), 1)

            # Summary strings
            tot_str = "NUMBER OF PEOPLE: " + str(total_p)
            high_str = "RED ZONE: " + str(high_risk_p)
            low_str = "ORANGE ZONE: " + str(low_risk_p)
            safe_str = "GREEN ZONE: " + str(safe_p)
    
            # Rectangle and Text Overlay at the bottom
            sub_img = img[height-120:height-20, 0:500]
            black_rect = np.ones(sub_img.shape, dtype=np.uint8) * 8
            res = cv2.addWeighted(sub_img, 0.8, black_rect, 0.2, 1.0)
            img[height-120:height-20, 0:500] = res
    
            cv2.putText(img, tot_str, (10, height-75), font, 0.6, (255, 255, 255), 1)
            cv2.putText(img, safe_str, (300, height-75), font, 0.6, (0, 255, 0), 1)
            cv2.putText(img, low_str, (10, height-50), font, 0.6, (0, 120, 255), 1)
            cv2.putText(img, high_str, (300, height-50), font, 0.6, (0, 0, 150), 1)
    
            # Drawing bounding boxes based on risk
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
    
            if status[kk] == 1:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 0, 150), 2)
            elif status[kk] == 0:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
            else:
                cv2.rectangle(img, (x, y), (x+w, y+h), (0, 120, 255), 2)
    
            kk += 1

            # Drawing lines between close pairs
            for h in close_pair:
                cv2.line(img, tuple(h[0]), tuple(h[1]), (0, 255, 255), 2)

            
                
                
    cv2.imshow('image',img)
    if cv2.waitKey(1)& 0xFF==ord('q'):
        break
    #cv2.waitKey(1)
    # fourcc=cv2.VideoWriter_fourcc(*'MJPG')
    # output=cv2.VideoWriter('output.mp4',fourcc,20.0,(img.shape[1],img.shape[0]))
    # output.write(img)
cap.release()
# output.release()
cv2.destroyAllWindows()
                
                
        
                
            

True
idf,person [0] [0]
True
idf,person [1 0] [1, 0]
True
True
idf,person [0] [0]
True
idf,person [0 1 3 2] [0, 1, 3, 2]
True
idf,person [2 1 4 3 5 0] [2, 1, 4, 3, 5, 0]
True
idf,person [1 4 2 0] [1, 4, 2, 0]
True
idf,person [3 4 7 0 5 1 6] [3, 4, 7, 0, 5, 1, 6]
True
idf,person [3 4 6 0 1] [3, 4, 6, 0, 1]
True
idf,person [1 2] [1, 2]
True
idf,person [1] [1]
True
idf,person [2 4 0 1 3] [2, 4, 0, 1, 3]
True
idf,person [4 7 5 1 6 2 3 0] [4, 7, 5, 1, 6, 2, 3, 0]
True
idf,person [6 5 7 0 9 3 2 1 8] [6, 5, 7, 0, 9, 3, 2, 1, 8]
True
idf,person [1 2 0] [1, 2, 0]
True
True
True
idf,person [0] [0]
True
idf,person [0 1] [0, 1]
True
idf,person [1 0] [1, 0]
True
idf,person [0] [0]
True
idf,person [0] [0]
True
idf,person [0 1] [0, 1]
True
idf,person [5 4 0 2 3] [5, 4, 0, 2, 3]
True
idf,person [4 3 2 1] [4, 3, 2, 1]
True
idf,person [0 1] [0, 1]
True
idf,person [0] [0]
True
idf,person [0] [0]
True
idf,person [0] [0]
True
idf,person [0] [0]
True
idf,person [0] [0]
True
idf,person [2 1 0] [2, 1, 0]
True